# Assignment: SQL Notebook for Peer Assignment

### Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

## Download the datasets

In [1]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Connect to the database

In [2]:
%load_ext sql

In [3]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_data1.db

In [5]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [6]:
# DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [7]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Task 1

Display the names of the unique launch sites in the space mission

In [10]:
import sqlite3
import pandas as pd
import prettytable

# Connect to the database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Assuming the data has already been loaded into the SPACEXTBL table
# Task 1: Display the names of the unique launch sites
query = "SELECT DISTINCT Launch_Site FROM SPACEXTBL;"
cur.execute(query)

# Fetch results
unique_launch_sites = cur.fetchall()

# Display results in a readable format using PrettyTable
table = prettytable.PrettyTable(["Unique Launch Sites"])
for site in unique_launch_sites:
    table.add_row(site)

print(table)

# Close the connection
con.close()


+---------------------+
| Unique Launch Sites |
+---------------------+
|     CCAFS LC-40     |
|     VAFB SLC-4E     |
|      KSC LC-39A     |
|     CCAFS SLC-40    |
+---------------------+


## Task 2

Display 5 records where launch sites begin with the string 'CCA

In [11]:
import sqlite3
import prettytable

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Task 2: Display 5 records where launch sites begin with 'CCA'
query = """
SELECT * 
FROM SPACEXTBL 
WHERE Launch_Site LIKE 'CCA%' 
LIMIT 5;
"""
cur.execute(query)

# Fetch the results
records = cur.fetchall()

# Display results in a readable format using PrettyTable
# Assuming SPACEXTBL has columns, adjust as necessary
column_names = [description[0] for description in cur.description]  # Get column names dynamically
table = prettytable.PrettyTable(column_names)

for record in records:
    table.add_row(record)

print(table)

# Close the connection
con.close()


+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
|    Date    | Time (UTC) | Booster_Version | Launch_Site |                            Payload                            | PAYLOAD_MASS__KG_ |   Orbit   |     Customer    | Mission_Outcome |   Landing_Outcome   |
+------------+------------+-----------------+-------------+---------------------------------------------------------------+-------------------+-----------+-----------------+-----------------+---------------------+
| 2010-06-04 |  18:45:00  |  F9 v1.0  B0003 | CCAFS LC-40 |              Dragon Spacecraft Qualification Unit             |         0         |    LEO    |      SpaceX     |     Success     | Failure (parachute) |
| 2010-12-08 |  15:43:00  |  F9 v1.0  B0004 | CCAFS LC-40 | Dragon demo flight C1, two CubeSats, barrel of Brouere cheese |         0         | 

## Task 3

Display the total payload mass carried by boosters launched by NASA (CRS)

In [18]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Task 3: Display the total payload mass carried by boosters launched by NASA (CRS)
query = """
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM SPACEXTBL
WHERE Customer LIKE '%NASA (CRS)%';
"""
cur.execute(query)

# Fetch the result
result = cur.fetchone()

# Display the result
if result[0] is not None:
    print(f"Total Payload Mass carried by NASA (CRS) boosters: {result[0]} kg")
else:
    print("No data found for boosters launched by NASA (CRS).")

# Close the connection
con.close()


Total Payload Mass carried by NASA (CRS) boosters: 48213 kg


## Task 4

Display average payload mass carried by booster version F9 v1.1


In [20]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Display average payload mass carried by booster version 'F9 v1.1'
query = """
SELECT AVG(PAYLOAD_MASS__KG_) AS Average_Payload_Mass
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1';
"""
cur.execute(query)

# Fetch the result
result = cur.fetchone()

# Display the result
if result[0] is not None:
    print(f"Average Payload Mass carried by booster version 'F9 v1.1': {result[0]:.2f} kg")
else:
    print("No data found for booster version 'F9 v1.1'.")

# Close the connection
con.close()


Average Payload Mass carried by booster version 'F9 v1.1': 2928.40 kg


## Task 5

List the date when the first succesful landing outcome in ground pad was acheived.

In [21]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  Find the date of the first successful landing outcome on a ground pad
query = """
SELECT MIN(Date) AS First_Successful_Landing
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (ground pad)';
"""
cur.execute(query)

# Fetch the result
result = cur.fetchone()

# Display the result
if result[0] is not None:
    print(f"The first successful landing outcome on a ground pad was achieved on: {result[0]}")
else:
    print("No successful landing outcome on a ground pad found in the data.")

# Close the connection
con.close()


The first successful landing outcome on a ground pad was achieved on: 2015-12-22


## Task 6

List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [22]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  List the names of boosters with 'Success (drone ship)' and payload mass between 4000 and 6000
query = """
SELECT Booster_Version
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ > 4000
  AND PAYLOAD_MASS__KG_ < 6000;
"""
cur.execute(query)

# Fetch all matching results
results = cur.fetchall()

# Display the results
if results:
    print("Boosters with 'Success (drone ship)' and payload mass between 4000 and 6000 kg:")
    for row in results:
        print(row[0])
else:
    print("No boosters found with 'Success (drone ship)' and payload mass between 4000 and 6000 kg.")

# Close the connection
con.close()


Boosters with 'Success (drone ship)' and payload mass between 4000 and 6000 kg:
F9 FT B1022
F9 FT B1026
F9 FT  B1021.2
F9 FT  B1031.2


## Task 7

List the total number of successful and failure mission outcomes

## Task 8

List the names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [23]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  List the total number of successful and failed mission outcomes
query = """
SELECT Mission_Outcome, COUNT(*) AS Total_Count
FROM SPACEXTBL
GROUP BY Mission_Outcome;
"""
cur.execute(query)

# Fetch all results
results = cur.fetchall()

# Display the results
if results:
    print("Total number of mission outcomes:")
    for row in results:
        print(f"{row[0]}: {row[1]}")
else:
    print("No mission outcomes found in the data.")

# Close the connection
con.close()


Total number of mission outcomes:
Failure (in flight): 1
Success: 98
Success : 1
Success (payload status unclear): 1


In [24]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  List the names of Booster_Version that carried the maximum payload mass using a subquery
query = """
SELECT Booster_Version
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTBL
);
"""
cur.execute(query)

# Fetch all matching results
results = cur.fetchall()

# Display the results
if results:
    print("Booster versions that carried the maximum payload mass:")
    for row in results:
        print(row[0])
else:
    print("No data found for maximum payload mass.")

# Close the connection
con.close()


Booster versions that carried the maximum payload mass:
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2 
F9 B5 B1058.3 
F9 B5 B1051.6
F9 B5 B1060.3
F9 B5 B1049.7 


## Task 9

List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [25]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  Query to display month names, failure landing outcomes on a drone ship, booster versions, and launch sites for 2015
query = """
SELECT 
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTBL
WHERE Landing_Outcome LIKE 'Failure (drone ship)'
  AND substr(Date, 1, 4) = '2015';
"""
cur.execute(query)

# Fetch all matching results
results = cur.fetchall()

# Display the results
if results:
    print("Records for failure landing outcomes on a drone ship in 2015:")
    print("Month | Landing Outcome | Booster Version | Launch Site")
    for row in results:
        print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]}")
else:
    print("No records found for failure landing outcomes on a drone ship in 2015.")

# Close the connection
con.close()


Records for failure landing outcomes on a drone ship in 2015:
Month | Landing Outcome | Booster Version | Launch Site
01 | Failure (drone ship) | F9 v1.1 B1012 | CCAFS LC-40
04 | Failure (drone ship) | F9 v1.1 B1015 | CCAFS LC-40


## Task 10

Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [26]:
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

#  Rank the count of landing outcomes between the given dates
query = """
SELECT 
    Landing_Outcome, 
    COUNT(*) AS Outcome_Count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;
"""
cur.execute(query)

# Fetch all matching results
results = cur.fetchall()

# Display the results
if results:
    print("Ranking of landing outcomes between 2010-06-04 and 2017-03-20:")
    print("Landing Outcome | Count")
    for row in results:
        print(f"{row[0]} | {row[1]}")
else:
    print("No landing outcomes found between the specified dates.")

# Close the connection
con.close()


Ranking of landing outcomes between 2010-06-04 and 2017-03-20:
Landing Outcome | Count
No attempt | 10
Success (drone ship) | 5
Failure (drone ship) | 5
Success (ground pad) | 3
Controlled (ocean) | 3
Uncontrolled (ocean) | 2
Failure (parachute) | 2
Precluded (drone ship) | 1
